In [4]:
#imports
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as ec
from collections import deque
from selenium.webdriver.common.keys import Keys
import time

In [5]:
#Chromedriver
browser = webdriver.Chrome()
browser.get('https://web.telegram.org/#/im')

In [7]:
#Lets the user select groups to track
def selectGroups(groupChats):
    index = 1
    for chat in groupChats:
        title= chat.find_elements_by_css_selector("div.im_dialog_peer")[0].text
        print(str(index) + '. ' + title)
        index+=1
    
    #once chats are printed, ask user for csv of indexes of groupchats they want to track
    listOfIndexOfChats = []
    while True:
        try:
            groupNumber = int(input("Please input the number of"+
                                    " the groupchat you want to track (any non-number to proceed): "))
            listOfIndexOfChats.append(groupNumber)
        except ValueError:
            break
            
    #now that we have the group indeces, use global variable to change indeces to chatNames
    userChats = []
    for index in listOfIndexOfChats:
        userChats.append(groupChats[index-1])
    print('You have selected the following groups: ')
    #return userChats
    return userChats
    
#________________________________________________________________________#
def selectHomeGroup(groupChats):
    index = int(input("Please input the number of the Home Group: "))
    homeGroup = groupChats[index-1].find_elements_by_css_selector("div.im_dialog_peer")[0].text
    print('Selected '+ homeGroup+' as homegroup.')
    return homeGroup

#________________________________________________________________________#
#Creates dictionairy of last messages in each group
def initializeDict(groupChats):
    lastMessageDict = {}
    for chat in groupChats:
        chat.click()
        selectedGroupTitle = chat.find_elements_by_css_selector("div.im_dialog_peer")[0].text
        while True:
            try:
                messages = browser.find_elements_by_class_name("im_message_text")
                textMessages = []
                #change array of WebElements to array of strings. easier to work with, avoid errors
                for webElement in messages:
                    textMessages.append(webElement.text)
                #for some reason there were empty strings being grabbed by selenium. removing them here    
                textMessages = list(filter(None, textMessages))
                key_gc = selectedGroupTitle
                lastMessageDict[key_gc] = textMessages[-1]
                print('Loaded ' + selectedGroupTitle + ': '+ lastMessageDict[key_gc])
                break
            except:
                print(selectedGroupTitle + 'did not load yet, retrying.')
    print(lastMessageDict)
    print('Recorded most recent messages in every group. New messages will be pushed to homegroup.')
    return lastMessageDict
#________________________________________________________________________#
#groupQueueBuilder
def groupQueueBuilder(trackedChats):
    for groupChat in trackedChats:
        oneBadgeArray = groupChat.find_elements_by_css_selector("span.im_dialog_badge.badge")
#         TO DO : make it not take messages from HOME GROUP
        if(oneBadgeArray[0].text!=""):
            #get index and use this index in titlesArray
            oneTitleArray = groupChat.find_elements_by_css_selector("div.im_dialog_peer")
            if(oneTitleArray[0].text not in groupQueue):
                groupQueue.append(oneTitleArray[0].text)
    print(groupQueue)
    return groupQueue
#________________________________________________________________________#
#groupFinder - Takes in name of group finds it selects it, returns title of slected chat
def groupFinder(name, trackedChats):
    print('Finding '+ name)
    for chat in trackedChats:
        titleArray = chat.find_elements_by_css_selector("div.im_dialog_peer")
        if(titleArray[0].text == name):
            chat.click()
            print('Clicked on ' + name)
            return(titleArray[0].text)
    print('groupFinder Could not find group: '+ name)
    return 'noGroupError'
    
#________________________________________________________________________#
#Create push method to click on 'home chat' and paste and send it there.
# checks via dictionary which messages to push


def createMessageStack(selectedGroupTitle, lastMessageDict):
#     groupChatContainerElement = browser.find_element_by_class_name("im_history_messages_peer")
    messageStack = []
    messages = browser.find_elements_by_class_name("im_message_text")
    textMessages = []
    print('Loading '+str(len(messages))+' new messages')
    #change array of WebElements to array of strings. easier to work with, avoid errors
    for webElement in messages:
        if(len(webElement.text) > 1):
            print(webElement.text)
        textMessages.append(webElement.text)
    #for some reason there were empty strings being grabbed by selenium. removing them here    
    textMessages = list(filter(None, textMessages))
    #reversing list to make FIFO
    textMessages.reverse()
    for message in textMessages:
        if(message != lastMessageDict[selectedGroupTitle]):
            messageStack.append(message)
        else:
#             SHOULD RUN WHEN MESSAGES ARE PUSHED
            break
    print((str)(len(messageStack)) + ' messages Loaded')
    print(messageStack)
    return messageStack
 #________________________________________________________________________#
# Broadcast Method
def broadCast(messageStack, selectedGroupTitle, groupChats, homeGroup, dictionairy):
    #select home chat
    print('Sending...')
    print(messageStack)
    messageStack.reverse()
    for chat in groupChats:
        titleArray = chat.find_elements_by_css_selector("div.im_dialog_peer")
        if(titleArray[0].text == homeGroup):
            chat.click()
            break
    
    #once homechat is selected, send messages one by one.
    textBox = browser.find_elements_by_class_name('composer_rich_textarea')[0]
    for message in messageStack:        
        textBox.send_keys(selectedGroupTitle + ": " + message)
        textBox.send_keys(Keys.RETURN)
    if(len(messageStack) != 0):
        dictionairy[selectedGroupTitle] = messageStack[-1]
    #clean up after sending messages.
    messageStack.clear()
    print('Messages sent!')
    #print(messageStack)
    return 0
#________________________________________________________________________#
def fillGroupQueue(trackedChats):
    groupQueue = deque([])
    for webElement in trackedChats:
        titleArray = webElement.find_elements_by_css_selector("div.im_dialog_peer")
        groupQueue.append(titleArray[0].text)
    print('Time to check every group for missed messages')
    print(groupQueue)
    return groupQueue
        
#________________________________________________________________________#
#User prompt to run BotLoop
global READY
READY = False

while True:
    print('______________________________________________________________')
    print('READ DIRECTIONS CAREFULLY.')
    try:
        userPrompt = input("Please type 'ready' after signing in to continue. ")            
        
    except ValueError:
        print("You gotta type in 'ready' exactly like that, bud...")
        continue

    else:
        if(userPrompt == 'ready'):
            READY = True
            break
        else:
            continue

#________________________________________________________________________#
#BotLoop
#grabs all groupchats

# while True:
#     print('inside botLoop')
# try:
if (READY):
    #Create Runtime Variables inside here.
    groupChats = browser.find_elements_by_class_name("im_dialog_wrap") 
    groupQueue = deque([])
    trackedChats = selectGroups(groupChats)
    homeGroup = selectHomeGroup(groupChats)
    lastMessageDict = initializeDict(trackedChats)
    timeout = 120
    timeout_start = time.time()
    cleanupTimeout = time.time()
    # month timeout:
    # while time.time() < 1517278786 + 259200:
    while time.time() < timeout_start + timeout:
        if(len(list(groupQueue)) == 0):
            time.sleep(1)
            if(time.time() > cleanupTimeout + 10):
                groupQueue = fillGroupQueue(trackedChats)
                cleanupTimeout = time.time()
            else:
                groupQueue = groupQueueBuilder(trackedChats)
                if(len(list(groupQueue)) != 0):
                    cleanupTimeout = time.time()
                
        else:
            selectedGroupTitle = groupFinder(groupQueue.popleft(), trackedChats)
            messageStack = createMessageStack(selectedGroupTitle, lastMessageDict)
            broadCast(messageStack, selectedGroupTitle, groupChats,homeGroup, lastMessageDict)
            cleanupTimeout = time.time()

    print('Finished BotLoop')
    
    
        
        
            #get first in queue and cache / print all messages from it.
            #For our purpose, batch work scheduler would be better. **prioritizes active groupchats**
    # except:
        # pass



______________________________________________________________
READ DIRECTIONS CAREFULLY.
Please type 'ready' after signing in to continue. ready
1. WhaleClub (Bitcoin ONLY) Traders
2. The Coin Farm
3. Refereum
4. ICOcountdown.com - http://telegram.me/icocountdown
5. Telegram
6. Alt Whales
7. Jakub Nowak
8. FGCU CRYPTO CRIPS
9. PAID GROUP
10. Danny A
11. RefereumBot
12. Whale Group
13. Telecast
14. Anton Invar
15. jerry lin
Please input the number of the groupchat you want to track (any non-number to proceed): 1
Please input the number of the groupchat you want to track (any non-number to proceed): 2
Please input the number of the groupchat you want to track (any non-number to proceed): 3
Please input the number of the groupchat you want to track (any non-number to proceed): 4
Please input the number of the groupchat you want to track (any non-number to proceed): l
You have selected the following groups: 
Please input the number of the Home Group: 9
Selected PAID GROUP as homegroup.
Wh

deque(['WhaleClub (Bitcoin ONLY) Traders'])
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 153 new messages
I prefer lenses.
just joking... lots of projects are cool and will succeed
In a few days.:joy:
Let’s hope so ha
Should get lasered dude
i'll watch how Japan & Korea act in 3hours approx then can say my vision of strategy
as for now see buy orders at 8k & 7.5k & take-profits somewhere +1.5k above
x3
:laughing:
green lasers or red lasers 🙂
Problem with BTC is that whales a majority of it
Then make a noose.. and hang in there.:joy:
Prefer blue
my chart doesn't haver blue.... : \
Sheit
oh
BLUE MOON!!!!
other coins need to pass too long road to reach what BTC has reached already
🤣🤣🤣
Honestly, I see a great future for some crypto. But not for Bitcoin. Like I said, many many like 90% of the BTC is owned by a few unknown people. What will they do with it? Are they trustworthy not to crash it or something else. Will they give it to charity? Ha

yeah, hopefully higher, 16K?
I haven't done the analysis
Wow.. give me that at least..:grin:
there will be loads of ideas out once we confirm bottom
2 messages Loaded
['there will be loads of ideas out once we confirm bottom', 'Wow.. give me that at least..:grin:']
Sending...
['there will be loads of ideas out once we confirm bottom', 'Wow.. give me that at least..:grin:']
Messages sent!
deque(['WhaleClub (Bitcoin ONLY) Traders', 'The Coin Farm'])
Finding WhaleClub (Bitcoin ONLY) Traders
Clicked on WhaleClub (Bitcoin ONLY) Traders
Loading 163 new messages
just joking... lots of projects are cool and will succeed
In a few days.:joy:
Let’s hope so ha
Should get lasered dude
i'll watch how Japan & Korea act in 3hours approx then can say my vision of strategy
as for now see buy orders at 8k & 7.5k & take-profits somewhere +1.5k above
x3
:laughing:
green lasers or red lasers 🙂
Problem with BTC is that whales a majority of it
Then make a noose.. and hang in there.:joy:
Prefer blue
my chart d

easy
hehehe
Dayum so many btc buyers rn
Easy??
Does anyone know just how good digibyte is :scream: so underrated, one day  :rocket:
Take a look at XBY
Next bubble i really need to sell🤓 for now hodl and buy more
Mozsi charts are :fire::fire::fire::fire:
Xrp is keeping me alive along with rdd :flushed:
XRP is the tranny u bang and never tell anyone was the best expression for it I've heard
was at 5k
Can't remember who said it
Think it was brewmaster
lmao
Fake taxi
VIBE !!!
New all time high will come after super bowl commercial, but don’t be so quick to sell them. The big growth will come when YouTubers and analysts discover it after the Super Bowl
Whattttt?
u know that the dotcom bubble ended with superbowl adverts
Then sellers too lol
3 messages Loaded
['Then sellers too lol', 'u know that the dotcom bubble ended with superbowl adverts', 'Whattttt?']
Sending...
['Then sellers too lol', 'u know that the dotcom bubble ended with superbowl adverts', 'Whattttt?']
Messages sent!
deque(['Wh

Then make a noose.. and hang in there.:joy:
Prefer blue
my chart doesn't haver blue.... : \
Sheit
oh
BLUE MOON!!!!
other coins need to pass too long road to reach what BTC has reached already
🤣🤣🤣
Honestly, I see a great future for some crypto. But not for Bitcoin. Like I said, many many like 90% of the BTC is owned by a few unknown people. What will they do with it? Are they trustworthy not to crash it or something else. Will they give it to charity? Hahaha how’s that guys?
I have about $100k in XRP. Hence the far bigger loss
not too long, crypto moves fast
Don’t know if emoticons are allowed
If they keep running thisuch of bear market, then everyone will understand their strategy..
NOTE: FINEX has a BID order of 2276 BTC in the book at 7800 USD (stop loss Hunter)  btcusd long chart see and 5.min %20 down :)
dips are
diiiiiiiiiiiiiip
Nice
Where you bought it from?
More sauce
I'm buying it in buckets
Man you need to update.
Shit. I said alts, not shitcoins. Haha. Unlucky mate. Still. Yo

In [57]:
linkedGroupChatContainer = browser.find_element_by_class_name('im_history_loaded')
print(linkedGroupChatContainer)
groupChatContainerElement = linkedGroupChatContainer.find_elements_by_class_name("im_history_messages_peer")
print(groupChatContainerElement)
for element in groupChatContainerElement:
    if(element.get_attribute("class") == 'im_history_messages_peer'): 
        messages = element.find_elements_by_class_name("im_message_text")
        print(messages)
textMessages = []
print('Loading '+str(len(messages))+' new messages')
#change array of WebElements to array of strings. easier to work with, avoid errors
for webElement in messages:
#     if(len(webElement.text) > 1):
    print(webElement.text)
    textMessages.append(webElement.text)
#for some reason there were empty strings being grabbed by selenium. removing them here    
textMessages = list(filter(None, textMessages))
print(textMessages)

<selenium.webdriver.remote.webelement.WebElement (session="5d9da736fc5b6f665a55f08a75b50bc0", element="0.45238913357743926-1")>
[<selenium.webdriver.remote.webelement.WebElement (session="5d9da736fc5b6f665a55f08a75b50bc0", element="0.45238913357743926-2")>, <selenium.webdriver.remote.webelement.WebElement (session="5d9da736fc5b6f665a55f08a75b50bc0", element="0.45238913357743926-3")>, <selenium.webdriver.remote.webelement.WebElement (session="5d9da736fc5b6f665a55f08a75b50bc0", element="0.45238913357743926-4")>]
[<selenium.webdriver.remote.webelement.WebElement (session="5d9da736fc5b6f665a55f08a75b50bc0", element="0.45238913357743926-5")>, <selenium.webdriver.remote.webelement.WebElement (session="5d9da736fc5b6f665a55f08a75b50bc0", element="0.45238913357743926-6")>, <selenium.webdriver.remote.webelement.WebElement (session="5d9da736fc5b6f665a55f08a75b50bc0", element="0.45238913357743926-7")>, <selenium.webdriver.remote.webelement.WebElement (session="5d9da736fc5b6f665a55f08a75b50bc0", el